<a href="https://colab.research.google.com/github/monteroanibal/diplomado_GEOIA_IGAC2025/blob/main/unidad_2_3_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupyter Notebook de la sesión

---

# 1. Cargue de librerías

Para generar y manipular datos en arreglos matriciales.

In [1]:
import numpy as np

Para desplegar las animaciones de los arreglos matriciales.

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML

Para importar y visualizar datos ráster.

In [3]:
%%capture
!pip install rasterio
import rasterio as rs

from rasterio.plot import show

# 2. Juego de la Vida de John Conway

El siguiente fragmento de código es adaptado de la siguiente fuente: https://analyticadss.com/conways-game-of-life-with-examples-in-r-and-python/. Este código recrea el _Juego de la Vida_ de John Conway, en donde las celdas de una tesela regular en un periodo o generación determinada "cobran vida" (valor uno [1]) o "mueren" (valor cero [0]) dependiendo del mismo estado de vida o muerte de las teselas vecinas en el periodo o generación anterior.

Para iniciar, se especifican los valores que parametrizan el proceso:

*   Número de generaciones o periodos a generar
*   Tamaño de la tesela, que para este caso se estableción de forma cuadrada.
*   Tiempo entre para reproducir la animación de la simulación generada.

Cabe mencionar que las celdas también son de forma cuadrada y que el estado inicial de la simulación es aleatoria, de modo que se está tabajando con una cuadrícula.

Se invita al estudiante a realizar diferentes configuraciones y experimentos para que observe las simulaciones con sus diferentes patrones de comportamiento, del tiempo empleado e incluso de los límites computacionales del servidor prestado por Google.

In [4]:
generations = 20
grid_side_size = 100
ms_frame_interv = 200

A continuación se explica cada sentencia que hace parte del proceso con los respectivos comentarios.

In [5]:
# Se genera un arreglo con estado inicial aleatorio
grid = np.random.choice([0, 1], size=(grid_side_size, grid_side_size))

# Se inicializa una lista para almacenar el estado de las celdas en cada generacioon
grid_history = [grid]

# el siguiente bucle realiza una simulacioon del -Juego de la Vida-
for i in range(generations):
    # Se obtiene el estado actual de las celdas
    current_state = grid_history[i]

    # Inicializa el siguiente estado de las celdas con valor cero, no necesariamente es el definitivo
    next_state = np.zeros((grid_side_size, grid_side_size))

    # Se itera sobre cada celda de la tesela
    for x in range(grid_side_size):
        for y in range(grid_side_size):
            # Se obtiene el número de vecinos vivos para la celda actual
            neighbors = (current_state[max(x-1, 0):min(x+2, grid_side_size), max(y-1, 0):min(y+2, grid_side_size)]).sum() - current_state[x, y]

            # Se aplican las reglas del -Juego de la Vida- para determinar el prooximo estado de la celda
            if current_state[x, y] == 1:
                if neighbors < 2 or neighbors > 3:
                    next_state[x, y] = 0
                else:
                    next_state[x, y] = 1
            else:
                if neighbors == 3:
                    next_state[x, y] = 1
                else:
                    next_state[x, y] = 0

    # Se actualiza el estado de las celdas
    grid = next_state

    # Se guarda el estado de las celdas para la generacioon recieen calculada
    grid_history.append(grid)

# Se omite la generacioon cero para la visualizacioon
grid_history = grid_history[1:]

# Se genera la animacioon para visualizar la simulacioon generada
def update(frame_number, matrix_data, img):
  img.set_array(matrix_data[frame_number])
  return img,
fig, ax = plt.subplots(figsize=(6,6))
img = ax.imshow(grid_history[0], cmap='gray', animated=True)
plt.axis('off')
ani0 = anim.FuncAnimation(fig, update, frames=len(grid_history), fargs=(grid_history, img), interval=ms_frame_interv, blit=True)
plt.close()
HTML(ani0.to_jshtml())

Se genera el .html de la animación para la simulación obtenida.

In [6]:
ani0.save(filename='ConwaysGameOfLifeXampl.html', writer='html')

In [7]:
%%capture
!zip -r ConwaysGameOfLifeXampl_frames.zip /content/ConwaysGameOfLifeXampl_frames

# 3. Autómatas Celulares para modelar expansión urbana

A continuación, se exploran los rasters de la clasificación de expansión urbana para el municipio de Villavicencio en periodos de tres años, con el objetivo de observar los patrones que permiten deducir un posible comportamiento y posterior modelado de este fenómeno de expansión a partir de esta clasifiación del uso del suelo en diferentes periodos de tiempo.


Se descargan los respectivcos rasters.

In [8]:
%%capture
!wget https://github.com/monteroanibal/diplomado_GEOIA_IGAC2025/raw/refs/heads/main/Insumos/Unidad_2_3_3_2_Datos/rasters.zip
!unzip -o /content/rasters.zip
!rm /content/rasters.zip

Se extrae el arreglo de cada ráster para general el listado de arreglos con el que se realiza la animación para explorar .

In [9]:
_2015 = rs.open('/content/expansion2015.tif').read()[0]
_2018 = rs.open('/content/expansion2018.tif').read()[0]
_2021 = rs.open('/content/expansion2021.tif').read()[0]
_2024 = rs.open('/content/expansion2024.tif').read()[0]
_2027 = rs.open('/content/expansion2027esc0.tiff').read()[0]
_2030 = rs.open('/content/expansion2030esc0.tiff').read()[0]
_2033 = rs.open('/content/expansion2033esc0.tiff').read()[0]
villavo_history = [_2015, _2018, _2021, _2024, _2027, _2030, _2033]

Se confiugura el tiempo entre para reproducir la animación de la simulación generada.

In [10]:
ms_frame_interv = 500

Se genera la animación para visualizar las capas de expansión urbana.

In [11]:
def update(frame_number, matrix_data, img):
  img.set_array(matrix_data[frame_number])
  anyo = 2015+(frame_number*3)
  es_simu = np.where(anyo>2025,'\n (simulación)','')
  ax.set_xlabel(f"{anyo} {es_simu}")
  return img,
fig, ax = plt.subplots(figsize=(9,6))
img = ax.imshow(villavo_history[0], cmap='gray', animated=True)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax.get_yaxis().set_visible(False)
ani1 = anim.FuncAnimation(fig, update,  frames=len(villavo_history), fargs=(villavo_history, img), interval=ms_frame_interv, blit=True)
plt.close()
HTML(ani1.to_jshtml())

Se genera el .html de la animación obtenida.

In [12]:
ani1.save(filename='VillavoExpUrb.html', writer='html')

In [13]:
%%capture
!zip -r VillavoExpUrb_frames.zip /content/VillavoExpUrb_frames

A continuación se realiza el mismo ejercicio anterior haciendo _zoom_ en un área específica que permita observar el patrón de comportamiento de la expansión en las celdas de la cuadrícula.

In [14]:
villavo_his_rec = [rec[1900:2000,1850:1950] for rec in villavo_history]
ms_frame_interv = 500

Se genera la animación para visualizar el _zoom_.

In [15]:
def update(frame_number, matrix_data, img):
  img.set_array( matrix_data[frame_number] )
  anyo = 2015+(frame_number*3)
  es_simu = np.where(anyo>2025,'\n (simulación)','')
  ax.set_xlabel(f"{anyo} {es_simu}")
  return img,
fig, ax = plt.subplots(figsize=(9,6))
img = ax.imshow(villavo_his_rec[0], cmap='gray', animated=True)
plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
ax.get_yaxis().set_visible(False)
ani2 = anim.FuncAnimation(fig, update,  frames=len(villavo_his_rec), fargs=(villavo_his_rec, img), interval=ms_frame_interv, blit=True)
plt.close()
HTML(ani2.to_jshtml())

Se genera el .html de la animación obtenida.

In [16]:
ani2.save(filename='VillavoExpUrbZoom.html', writer='html')

In [17]:
%%capture
!zip -r VillavoexpUrbZoom_frames.zip /content/VillavoExpUrbZoom_frames